In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
Train=pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/train.csv')
Test= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/test.csv')
sample_submission= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/sample_submission.csv')

In [ ]:
#Filter BY CO2 emissions (metric tons per capita)
specific_string = 'CO2 emissions (metric tons per capita)'
filtered_rows = Train[Train['Indicator'] == specific_string]
filtered_rows

In [ ]:
#Drop Indicator Column
filtered_rows.drop(columns=['Indicator'], inplace=True)
filtered_rows

In [ ]:
#Drop Country Code Column
filtered_rows.drop(columns=['Country Code'], inplace=True)
filtered_rows

In [ ]:
#Covert Year Column in rows
melted_df = filtered_rows.melt(id_vars='Country Name', var_name='Year', value_name='CO2_Emissions')
melted_df

In [ ]:
#Sort Row in accending order by Year and Country Name
sorted_df = melted_df.groupby('Country Name').apply(lambda x: x.sort_values('Year')).reset_index(drop=True)
sorted_df.head(20)

In [ ]:
#All country name into unique_values
unique_values = melted_df['Country Name'].unique()
unique_values.shape

In [ ]:
new_value = 0.00
sorted_df = sorted_df.replace('..', new_value)

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")
warnings.resetwarnings()
total=0
j=0
for i in unique_values:
    afg = sorted_df[sorted_df['Country Name'] == i]
    # Define SARIMAX parameters
    order = (1, 1, 0)  # (p, d, q)
    seasonal_order = (1, 1, 0, 6)  # (P, D, Q, s)
    # Fit SARIMAX model
    model = SARIMAX(afg['CO2_Emissions'].astype(float), order=order, seasonal_order=seasonal_order)
    result = model.fit()

    # Predictions
    forecast_steps = 16  # Number of steps to forecast
    forecast = result.forecast(steps=forecast_steps)
    row_index = sample_submission[sample_submission.eq(i).any(axis=1)].index[0]


    new_values = [i, forecast[16], forecast[17],forecast[18], forecast[19], forecast[20],forecast[30]]
    sample_submission.loc[row_index] = new_values



In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.to_csv('submission.csv', index=False)